In [8]:
!pip install pdfplumber
!brew install freetype imagemagick
!brew install ghostscript

To reinstall 2.10.1, run `brew reinstall freetype`
To reinstall 7.0.9-26, run `brew reinstall imagemagick`
==> Downloading https://homebrew.bintray.com/bottles/ghostscript-9.50.catalina.b
==> Downloading from https://akamai.bintray.com/c1/c1e11a68fdd8b406979fc51791cb1
######################################################################## 100.0%
==> Pouring ghostscript-9.50.catalina.bottle.tar.gz
🍺  /usr/local/Cellar/ghostscript/9.50: 670 files, 82.3MB


In [ ]:
# 필요 데이터

# 진로 희망사항 ->  

# 창의적 체험활동
    #1학년
        #자율활동
        #동아리 활동
        #봉사 활동
        #진로 활동
    #2학년
        #자율활동
        #동아리 활동
        #봉사 활동
        #진로 활동
    #3학년
        #자율활동
        #동아리 활동
        #봉사 활동
        #진로 활동

# 세부 특기사항

# 봉사활동




data = {
    "진로희망사항": {
        "1학년":{
            "진로 희망": "",
            "희망 사유": "",
        },
        "2학년":{
            "진로 희망": "",
            "희망 사유": "",
        }
    },
    
    "창의적체험활동": {
        "1학년": {
            
            "자율활동":{

            },
            "동아리 활동":{

            },
            "봉사 활동":{

            },
            "진로 활동":{

            },
        },
        "2학년": {
            "자율활동":{

            },
            "동아리 활동":{

            },
            "봉사 활동":{

            },
            "진로 활동":{

            },
        },
        "3학년": {
            "자율활동":{

            },
            "동아리 활동":{

            },
            "봉사 활동":{

            },
            "진로 활동":{

            },
        },
    },
}


        

In [843]:
import pdfplumber
import pandas as pd




with pdfplumber.open("./testDoc4.pdf") as pdf:
    
    #first_page = pdf.pages[2]
       
    # 추출 이미지 확인
    #im = first_page.to_image(resolution=150)
    #im.draw_rects(first_page.extract_words())
    #im.save("./output", format="PNG")
    
    
    #print(getData_진로희망사항(table))
    
    봉사실적 = [] # 2020, 2016
    창의적체험활동상황 = [] #2020, 2016
    진로희망사항 = [] # 2020, 
    세부특기사항 = [] # 2020, 2016
    수상실적 = [] # 2020, 2016
    
    # 생기부 버전 읽기
    try:
        pageVersion = int(pdf.pages[0].extract_tables()[-2][0][1][:4])
    except:
        pageVersion = int(pdf.pages[0].extract_tables()[-1][0][1][:4])
        pass
        
    print(pageVersion)
    
    
    for pageIndex in range(len(pdf.pages)):
       
        #targetPage = pdf.pages[2]
        targetPage = pdf.pages[pageIndex]
        table = targetPage.extract_tables()
        
   
        for dataSets in table:
        
            if len(dataSets[0]) > 0:
                if dataSets[0][0] != None:
                    flagData = dataSets[0][0].replace(" ", "") #2020년 생기부 데이터 기준 파싱 위치
                    if flagData[2:] == '세부능력및특기사항':
                        세부특기사항.append(dataSets[1])
                    
                    # ==== 수상경력 데이터 파싱 부분 ====
                    
                    if flagData[2:] == '수상경력': # 해당 부분은 2016버전, 2020버전 모두 동작함.
                        [수상실적.append(index) for index in dataSets[2:]]
                        
                    if flagData == '수상명': # 2020 버전만 동작함
                        [수상실적.append(index) for index in dataSets[1:]]
                        
                    # ==== 진로희망사항 데이터 파싱 부분 ====
                        
                    if flagData[2:] == '진로희망사항' and pageVersion <= 2016: # 2016년 이하 버전에서만 동작
                        [진로희망사항.append([index[0], index[3], index[4]]) for index in dataSets[3:]]
                    
         
        
            if len(dataSets[0]) > 1:
                if dataSets[0][1] != None:
                    flagData = dataSets[0][1].replace(" ", "")
                    # ==== 창의적체험활동상황 데이터 파싱 부분 ====
                    
                    if  flagData == '창의적체험활동상황':              
                        [창의적체험활동상황.append(index) for index in dataSets[2:]]
                    
                    # ==== 봉사활동실적 데이터 파싱 부분 ====
                    
                    if flagData == '봉사활동실적':
                        [봉사실적.append(index) for index in dataSets[2:]]

                    
           
            if len(dataSets) > 1:
                if len(dataSets[0]) > 1 :
                    if dataSets[1][1] != None: 
                        # ==== 진로희망 데이터 파싱 부분 ====
                        pass
                        if dataSets[1][1].replace(" ", "")  == '진로희망': #2020년 버전만 동작함, 처음 항목 파싱
                            [진로희망사항.append(index) for index in dataSets[2:]]
                        
                        # =============================
                    
                    if dataSets[0][1] != None:
                        flagData = dataSets[0][1].replace(" ", "")
                        
                        # ==== 진로희망 데이터 파싱 부분 ====
                        
                        if flagData  == '진로희망': #2020년 버전만 동작
                            [진로희망사항.append(index) for index in dataSets[2:]]
                        
                        # ==== 생기부 수상경력 데이터 파싱 부분 ====
                        
                        if flagData == '수상명': # 2016 버전만 동작함, index 0번 데이터가 비어있음.
                            [수상실적.append(index) for index in dataSets[1:]]
                            
                        # ==== 진로희망 데이터 파싱 부분 ====
                        
                        if flagData == '특기또는흥미': # 2016 버전만 동작함. 편의를 위해 특기또는흥미로 검색하도록 설정함
                            [진로희망사항.append([index[0], index[3], index[4]]) for index in dataSets[2:]]
                            
    
    # 봉사실적 1, 3,
    # 창의적체험활동상황 1, 3
    # 진로희망사항 1, 3
    # 세부특기사항 1, 3
    # 수상실적 1, 3
    
    #for i in 진로희망사항:
    #    print(i)
        
    #print(진로희망사항)   
    
def dataNormalization_봉사실적():
    pass

def dataNormalization_창의적체험활동상황():
    pass

def dataNormalization_진로희망사항():
    pass

def dataNormalization_세부특기사항():
    pass

def dataNormalization_수상실적():
    pass
        
        
def formatData_Json_진로희망사항(raw_data_list):
    formatedData ={
       "1학년":{
            "진로희망": "",
            "희망사유": "",
        },
        "2학년":{
            "진로희망": "",
            "희망사유": "",
        } ,
        "3학년":{
            "진로희망": "",
            "희망사유": "",
        } ,
    } 
    for dataSet in raw_data_list:
        formatedData[dataSet[0] + "학년"]= {
            "진로희망": dataSet[1].replace("\n", ""),
            "희망사유": dataSet[2].replace("\n", "")
        }
    return formatedData

print(formatDataJson_진로희망사항(진로희망사항))

2016
{'1학년': {'진로희망': '교사', '희망사유': '가르치고 이해시키고 깨닫게 하는 것에 보람을 느낌.'}, '2학년': {'진로희망': '세무공무원', '희망사유': '경제와 경영에 관한 관심이 많아 꾸준한 독서를 수행하면서해당 영역에 대한 진로를 고민해왔음. 펀드 매니저에게 개인이나 기업이 투자하는 것은 결국 상호 신뢰를 바탕으로 이뤄진다고 생각하고 그 점에서 강한 매력을 느껴 펀드 매니저로서의 진로를 설정하게 되었다고상담에서 밝힘.'}, '3학년': {'진로희망': '펀드매니저, 세무공무원', '희망사유': "경제와 경영에 관심이 많아 평소 해당 분야와 관련된 도서와인터넷 검색을 통해 진로 탐색을 해왔음. '커리어넷'에서 펀드매니저 동영상을 보며 펀드매니저가 국내, 세계 경제에서 벌어지는 모든 산업에 대해 공부하고 이 산업이 흥할 것인지 망할"}, '학년': {'진로희망': '', '희망사유': '것인지 전망을 제시하는 직업이므로 다양한 분야에서 새로운지식을 끊임없이 얻을 수 있다는 점에서 흥미를 느껴 진로를설정하게 되었다고 밝혔음.'}}


['1', '자율활동', '113', '학교폭력과 신변안전(2017.03.15.)에서 친구와의 올바른 소통을 통해 좋\n은 친구를 사귀고 학교폭력을 예방하는 방법에 대해 알게 되었음.\n위기상황 대처법 및 생존법(2017.03.21.)  안전교육 시간에 국민들이 실천\n해야 할 사항에 대해 많은 관심을 가지고 교육에 적극 참여함. \n청소년기의 특징 교육(2017.04.04.)을 통하여 청소년기의 특징을 이해하\n고 가정 폭력에 대처하는 방법을 알게 됨.\n 에이즈 및 성매매 예방교육(2017.05.17.)을 통해 에이즈의 본질에 대하여\n알게 되었으며 에이즈보균자, 나환자들이 막연히 두려워하거나 피할 대상이\n아님을 알게 되었고, 건강에 유의할 것을 다짐함.\n백일장(2017.10.12)에서 시와 산문을 적어내면서 자신의 생각을 글로 적\n어내는 것이 생각보다 힘들고 중요하다는 점을 깨달음.\n마라톤대회(2017.10.17)에서 체력이 다소 부진한 친구를 도와 같이 완주\n하였음.\n융합수학창의작품대회(2017.10.20)에서  아포피시스7이라는  프로그램을\n익혀서 프랙탈아트 작품을 만들고 함수개념과 등비 수열 개념을 이해, 응용\n하여 작품을 완성해냄.\n학급별체험활동(2017.10.27)에서 놀이공원에서 탄 놀이기구의 물리법칙\n을 급우와 이야기하고 Unity3D를 이용하여 놀이기구를 프로그래밍으로 구\n현하였다. 이 과정에서 친구와 협업하여 자신의 재능을 응용하는 방법을 깨']
--2페이지--
['1', '', '', '달음.']
[None, '동아리활동', '24', "(IOT프로그래밍반)(10시간) 



[학년, 활동명, 시간, 내용"

학년 - > None: 같은 학년
None -> 학년: 새로운 학년


활동명 -> None: 같은 활동명
None -> 활동명: 새로운 활동명



In [829]:
index = ['1', '독서', '교사', '교사', '가르치고 이해시키고 깨닫게 하\n는 것에 보람을 느낌.']


['1', '교사', '가르치고 이해시키고 깨닫게 하\n는 것에 보람을 느낌.']